In [1]:
import zipfile
import datetime
import string
import math
import os

import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.model_selection

import keras_ocr
from keras.preprocessing import image
from keras.models import load_model
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import random
import os
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.densenet import DenseNet121, preprocess_input
import matplotlib.pyplot as plt
import keras_ocr
from keras_ocr.detection import Detector
import numpy as np
import cv2
from keras_ocr.tools import read

data_dir = '.'
alphabet = string.digits + string.ascii_letters + '!?. '

recognizer_alphabet = ''.join(sorted(set(alphabet.lower())))
fonts = keras_ocr.data_generation.get_fonts(alphabet=alphabet, cache_dir=data_dir)

backgrounds = keras_ocr.data_generation.get_backgrounds(cache_dir=data_dir)


Looking for ./fonts.zip


Filtering fonts.: 100%|██████████████████████████████████████████████████████████████| 2746/2746 [00:10<00:00, 261.36it/s]


Looking for ./backgrounds.zip


In [91]:
from PIL import Image, ImageDraw, ImageFont
import random
import os
import csv  
def calculate_luminance(color):
    # Calculate luminance based on the perceived brightness of RGB values
    return 0.299 * color[0] + 0.587 * color[1] + 0.114 * color[2]

def get_contrast_color(background_color):
    # Choose black or white as the text color based on background luminance
    luminance = calculate_luminance(background_color)
    return (255, 255, 255) if luminance < 128 else (0, 0, 0)

# Set larger image dimensions
width, height = 1200, 900

# Number of images to generate
num_images = 1000

# List of words to include in the images
word_list = ["cat", "dog","potato","blue","red","green","size","lord"]


output_dir = "images_no_rotation"
os.makedirs(output_dir, exist_ok=True)

font_paths = []
for font_path in fonts:
    font_paths.append(font_path)
background_paths = []
for background_path in backgrounds:
    background_paths.append(background_path)
reference_font_size = 1200
csv_file_path = "./image_labels_no_rotation.csv"
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Image_Path', 'Selected_Word'])

    for i in range(num_images):
        # Create a blank image
        image = Image.new("RGB", (width, height), "white")

        background_path = random.choice(background_paths)
        background = Image.open(background_path)
        background = background.resize((width, height))  # Resize the background to match the desired dimensions

        # Create a blank image with the background
        image = Image.new("RGB", (width, height), "white")
        image.paste(background, (0, 0))
        draw = ImageDraw.Draw(image)



        # Randomly select font size and font path
        font_size = 400
        font_path = random.choice(font_paths)
        selected_word = random.choice(word_list)
        reference_font = ImageFont.truetype(font_path, reference_font_size)
        reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
        scaling_factor = reference_font_size / max(reference_text_width, reference_text_height)
        current_font_size = int(reference_font_size * scaling_factor)
        font = ImageFont.truetype(font_path, current_font_size)

        # Calculate the position to center the word
        text_width, text_height = draw.textsize(selected_word, font)
        x = (width - text_width) // 2
        y = (height - text_height) // 2

        # Randomly rotate and scale the text
        rotation_angle = random.uniform(-15, 15)
        scale_factor = random.uniform(0.8, 1.2)
        #X, y = font.getsize(selected_word)
        text_layer = Image.new("RGBA", (width, height), (255, 255, 255, 0))
        text_draw = ImageDraw.Draw(text_layer)

            # Get the background color at the text position
        background_color = image.getpixel((x, y))

        # Choose the contrasting text color based on background color luminance
        text_color = get_contrast_color(background_color)

        text_draw.text((x, y), selected_word, font=font, fill=text_color)

        rotated_text = text_layer.rotate(rotation_angle, resample=Image.BICUBIC, expand=True)
        scaled_text = rotated_text.resize((int(text_width * scale_factor), int(text_height * scale_factor)))
        x_centered = (width - scaled_text.width) // 2
        y_centered = (height - scaled_text.height) // 2
        image.paste(scaled_text, (x_centered, y_centered), scaled_text)


        # Save the generated image
        image_path = os.path.join(output_dir, f"generated_image_{i + 1}.png")
        plt.imshow(image)
        image.save(image_path)
        csv_writer.writerow([image_path, selected_word])


print(f"{num_images} images generated and saved to {output_dir}.")

/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:59: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:65: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(selected_word, font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:59: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:65: DeprecationWarning: textsize is

/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:59: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:65: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(selected_word, font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:59: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:65: DeprecationWarning: textsize is

/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:59: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:65: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(selected_word, font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:59: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:65: DeprecationWarning: textsize is

/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:59: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:65: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(selected_word, font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:59: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:65: DeprecationWarning: textsize is

/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:59: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:65: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(selected_word, font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:59: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:65: DeprecationWarning: textsize is

...

/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:59: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:65: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(selected_word, font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:59: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  reference_text_width, reference_text_height = draw.textsize(selected_word, font=reference_font)
/var/folders/tq/3y55w7sx65sc1bhw_h244nqr0000gn/T/ipykernel_66153/3275292832.py:65: DeprecationWarning: textsize is

1000 images generated and saved to images_no_rotation.
Error in callback <function _draw_all_if_interactive at 0x116dd4360> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x29a3a9440> (for post_execute):


KeyboardInterrupt: 

In [123]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Load data from the CSV file using pandas
csv_file_path = "./image_labels_no_rotation.csv"
df = pd.read_csv(csv_file_path)

# Split the data into training, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

# Create data generators for training, validation, and test
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Image_Path',
    y_col='Selected_Word',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='Image_Path',
    y_col='Selected_Word',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical' 
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Image_Path',
    y_col='Selected_Word',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical' 
)



Found 800 validated image filenames belonging to 8 classes.
Found 100 validated image filenames belonging to 8 classes.
Found 100 validated image filenames belonging to 8 classes.


In [97]:
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.densenet import DenseNet121, preprocess_input


model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128,128, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5)) 

model.add(Dense(8, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=25,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)


Epoch 1/25
25/25 [==============================] - 20s 806ms/step - loss: 2.1514 - accuracy: 0.1375 - val_loss: 2.0529 - val_accuracy: 0.1875
Epoch 2/25
25/25 [==============================] - 19s 771ms/step - loss: 2.0265 - accuracy: 0.2138 - val_loss: 1.8656 - val_accuracy: 0.2292
Epoch 3/25
25/25 [==============================] - 19s 771ms/step - loss: 1.7811 - accuracy: 0.3325 - val_loss: 1.5990 - val_accuracy: 0.3750
Epoch 4/25
25/25 [==============================] - 19s 767ms/step - loss: 1.4630 - accuracy: 0.4650 - val_loss: 1.0496 - val_accuracy: 0.6979
Epoch 5/25
25/25 [==============================] - 20s 778ms/step - loss: 0.8984 - accuracy: 0.6825 - val_loss: 0.7466 - val_accuracy: 0.8125
Epoch 6/25
25/25 [==============================] - 19s 770ms/step - loss: 0.6526 - accuracy: 0.7875 - val_loss: 0.5259 - val_accuracy: 0.8229
Epoch 7/25
25/25 [==============================] - 19s 772ms/step - loss: 0.4479 - accuracy: 0.8450 - val_loss: 0.5710 - val_accuracy: 0.8125

KeyboardInterrupt: 

In [98]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

3/3 [==============================] - 2s 676ms/step - loss: 0.4919 - accuracy: 0.8542
Test Accuracy: 85.42%


In [117]:
import matplotlib.pyplot as plt
import keras_ocr
from keras_ocr.detection import Detector
import numpy as np
import cv2
from keras_ocr.tools import read

test_image_path = "./test_images/cat.jpg"


# Create a custom text detector/ not being used 
custom_detector = Detector(weights='clovaai_general')

# Create a custom OCR pipeline with only the text detector
custom_pipeline = keras_ocr.pipeline.Pipeline()
images_not = ["https://upload.wikimedia.org/wikipedia/commons/b/bd/Army_Reserves_Recruitment_Banner_MOD_45156284.jpg",
              'https://upload.wikimedia.org/wikipedia/commons/b/b4/EUBanana-500x112.jpg']

# image url
image_urls = [
    "./images_/generated_image_971.png"
]

# read and reszize 
images = [cv2.resize(read(url), (500, 500)) for url in image_urls]

# Use the custom detector to detect text in the images
detections = custom_pipeline.detector.detect(images)


Looking for /Users/zacharyweisenbloom/.keras-ocr/craft_mlt_25k.h5
Looking for /Users/zacharyweisenbloom/.keras-ocr/craft_mlt_25k.h5
Looking for /Users/zacharyweisenbloom/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 1s 738ms/step


In [118]:
import cv2

for i, detection in enumerate(detections):
    for j, text_info in enumerate(detection):
        # Extract coordinates
        x_min, y_min = map(int, text_info.min(axis=0))
        x_max, y_max = map(int, text_info.max(axis=0))


        text_region = images[i][y_min:y_max, x_min:x_max]
        cv2.imwrite(f"text_region_{i + 1}_{j + 1}.png", text_region)


In [7]:

predictions = []

for i, detection in enumerate(detections):
    for j, text_info in enumerate(detection):
        # Extract coordinates
        x_min, y_min = map(int, text_info.min(axis=0))
        x_max, y_max = map(int, text_info.max(axis=0))

        # Crop and save the text region
        text_region = images[i][y_min:y_max, x_min:x_max]

        # Preprocess the image based on your model's requirements
        # You may need to normalize pixel values, reshape, or apply other preprocessing steps
        preprocessed_text_region = preprocess_input(text_region)


        # Make predictions
        prediction = model.predict(preprocessed_text_region)

        # Append the prediction to the list
        predictions.append(prediction)

# `predictions` now contains the model predictions for each text region

ValueError: in user code:

    File "/Users/zacharyweisenbloom/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 2416, in predict_function  *
        return step_function(self, iterator)
    File "/Users/zacharyweisenbloom/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 2401, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/zacharyweisenbloom/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 2389, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/zacharyweisenbloom/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 2357, in predict_step
        return self(x, training=False)
    File "/Users/zacharyweisenbloom/anaconda3/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/zacharyweisenbloom/an

In [177]:
for i, detection in enumerate(detections):
    for j, text_info in enumerate(detection):
        # Extract coordinates
        x_min, y_min = map(int, text_info.min(axis=0))
        x_max, y_max = map(int, text_info.max(axis=0))

        # Crop and resize the text region
        text_region = cv2.resize(images[i][y_min:y_max, x_min:x_max], (128, 128))

        # Preprocess the image based on your model's requirements
        # You may need to normalize pixel values, reshape, or apply other preprocessing steps
        preprocessed_text_region = preprocess_input(text_region)

        # Add a batch dimension (if required by your model)
        preprocessed_text_region = np.expand_dims(preprocessed_text_region, axis=0)

        # Make predictions
        prediction = model.predict(preprocessed_text_region)

        # Find the index of the maximum value in the prediction array
        predicted_class_index = np.argmax(prediction)

        # Get the corresponding class label
        predicted_class_label = word_list[predicted_class_index]

        # Print the result
        print(f"Prediction for text region {i}, instance {j}: {predicted_class_label}")

1/1 [==============================] - 0s 20ms/step
Prediction for text region 0, instance 0: red


In [121]:
from keras.preprocessing import image
from keras.models import load_model
import numpy as np

# Step 1: Preprocess the image
img_path = './images_no_rotation/generated_image_558.png'
img = image.load_img(img_path, target_size=(128, 128))  # Adjust target_size based on your model's input size
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize pixel values

# Step 2: Load the model

# Step 3: Make predictions
predictions = model.predict(img_array)
print(predictions)
# Step 4: Post-process predictions (example: get the class with the highest probability)
predicted_class = np.argmax(predictions)
print(f"Predicted Class: {predicted_class}")

1/1 [==============================] - 0s 14ms/step
[[2.6330570e-11 2.1770115e-17 4.2934063e-12 1.3641560e-14 1.0000000e+00
  3.1663331e-12 5.8919770e-12 3.0972043e-13]]
Predicted Class: 4
